In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Data Analysis

The central problem that I will be solving for this homework will be the classificiation of varius handwritten number raning zero to 9.

In [ ]:
## Reading the dataset

import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Datasets/handwritten_numbers_dataset.csv')
df.describe()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
count,42000.000000,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,...,42000.000000,42000.000000,42000.000000,42000.00000,42000.000000,42000.000000,42000.0,42000.0,42000.0,42000.0
mean,4.456643,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.219286,0.117095,0.059024,0.02019,0.017238,0.002857,0.0,0.0,0.0,0.0
std,2.887730,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.312890,4.633819,3.274488,1.75987,1.894498,0.414264,0.0,0.0,0.0,0.0
min,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
25%,2.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
50%,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
75%,7.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
max,9.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,254.000000,254.000000,253.000000,253.00000,254.000000,62.000000,0.0,0.0,0.0,0.0


In [137]:
#determine the number of instances of each label

# determine the distribution of positive to negative classifications
for l in range(10):
  n = df['label'].value_counts()[l]
  print("Label {}:, {:.2f}%".format(l, n/df.shape[0] * 100))

Label 0:, 9.84%
Label 1:, 11.15%
Label 2:, 9.95%
Label 3:, 10.36%
Label 4:, 9.70%
Label 5:, 9.04%
Label 6:, 9.85%
Label 7:, 10.48%
Label 8:, 9.67%
Label 9:, 9.97%


Because the differences between the proportion of the dataset that each label makes up is not very much, being at most 2 percent, I see no need to try and create a more even distribution for training purposes. In fact, to ignore some data for training could be detrimental to the models ability.

In [ ]:
#split the data

ratio = 0.8 #the proportion of the full dataset dedicated
rows = len(df)  #total number of rows in the full dataset

split = int(rows * ratio)

# Split data into testing and training data
df_train = df[0:split]
df_test = df[split:]

In [153]:
import torch
import numpy as np
import math

#generates a tensor of dimension nin initialized with Glorot Scheme
def glorot(nin, nout):
    r = math.sqrt(6/(nin + nout))
    return np.random.uniform(-r, r, (nin, nout))

#ReLu Function
def relu(x):
    return max(0.0, x)

#sigmoid function
def sigmoid(x):
  try:
    #print(f"sigmoid x: {x}", flush = True)
    return 1/(1 + math.exp(-x))
  except OverflowError:
    if(x > 0):
      return 1
    else:
      return 0

#derivative of the ReLu function
def dRelu(x):
    if(x > 1):
        return 1
    else: #the derivative is actually 0 at x=0, but just assume it is 0
        return 0

#derivative of the sigmoid function
def dSigmoid(x):
    return sigmoid(x) * (1 - sigmoid(x))

class NeurelNetwork:

    #n0 : input layer dimension
    #n1, n2: hidden layer dimensions
    #n3: output layer dimension
    def __init__(self, n0, n1, n2, n3):
      #initalize the weight
      self.W1 = glorot(n1, n0) # hidden layer
      self.W2 = glorot(n2, n1) # hidden layer
      self.W3 = glorot(n3, n2) # output layer

      #intialize the biases
      self.b1 = np.zeros(n1, dtype=np.float32) # hidden laye 1
      self.b2 = np.zeros(n2, dtype=np.float32) # hidden layer 2
      self.b3 = np.zeros(n3, dtype=np.float32) # output layer

      #initalize variables for Adam Optimizer
      self.timeStep = 1

      self.nuP_W1 = np.zeros((n1, n0))
      self.hP_W1 = np.zeros((n1, n0))

      self.nuP_W2 = np.zeros((n2, n1))
      self.hP_W2 = np.zeros((n2, n1))

      self.nuP_W3 = np.zeros((n3, n2))
      self.hP_W3 = np.zeros((n3, n2))

      self.nuP_b1 = np.zeros(n1)
      self.hP_b1 = np.zeros(n1)

      self.nuP_b2 = np.zeros(n2)
      self.hP_b2 = np.zeros(n2)

      self.nuP_b3 = np.zeros(n3)
      self.hP_b3 = np.zeros(n3)

    def h1(self, a):
      return np.vectorize(relu)(a) #vectorize the ReLu function and apply it to the matrix

    def h2(self, a):
      return np.vectorize(relu)(a) #vectorize the ReLu function and apply it to the matrix

    def h3(self, a):
      return np.vectorize(sigmoid)(a) #vectorize the ReLu function and apply it to the matrix

    def dh1(self, a):
      return np.vectorize(dRelu)(a)

    def dh2(self, a):
      return np.vectorize(dRelu)(a)

    def dh3(self, a):
      return np.vectorize(dSigmoid)(a)

    def loss(self, Xbatch, Ybatch):
      #compute the output of the network using Xbatch

      #print(f"MATRIX: W1: {self.W1.shape}, XBATCH: {Xbatch.shape}, B1: {self.b1.shape}", flush = True)

      forward_a1 = np.matmul(self.W1, self.Xbatch) + self.b1[:, np.newaxis]
      forward_o1 = self.h1(forward_a1) #apply the ReLu function

      #print(f"MATRIX: W2: {self.W2.shape}, o1: {self.o1.shape}, B2: {self.b2.shape}", flush = True)

      forward_a2 = np.matmul(self.W2,  forward_o1) + self.b2[:, np.newaxis]
      forward_o2 = self.h2(forward_a2) #apply the ReLu function

      #print(f"MATRIX: W3: {self.W3.shape}, o2: {self.o2.shape}, B1: {self.b3.shape}", flush = True)

      forward_a3 = np.matmul(self.W3,  forward_o2) + self.b3[:, np.newaxis]
      Ypreds= self.h2(forward_a3) #apply the Sigmoid function

      #Retrieve the dimensions of the Xbatch
      d, m = Xbatch.shape

      #compute the loss using the Xbatch, and Ybatch
      f = lambda x, y : -(x * np.log(y) + (1 - x) * np.log(1 - y)) #negative log likely-hood

      #print(f"YBATCH: {Ybatch.shape} YPREDS: {Ypreds.shape}")


      return np.mean(np.vectorize(f)(Ybatch, Ypreds))

    def dloss(self, Ypreds):
        #compute the derivative of the loss function with respect to Y (x-y/ y-y^2)
        df = lambda x, y : (x-y)/max((y-y**2), 10e-8)

        return np.vectorize(df)(self.Ybatch, Ypreds)

    def forward(self, Xbatch):
      #store the Xbatch as an instance
      self.Xbatch = np.copy(Xbatch)

      #print(f"MATRIX: W1: {self.W1.shape}, XBATCH: {Xbatch.shape}, B1: {self.b1.shape}", flush = True)

      #store the activations and outputs as instance variables
      self.a1 = np.matmul(self.W1, self.Xbatch) + self.b1[:, np.newaxis]
      self.o1 = self.h1(self.a1) #apply the ReLu function

      #print(f"MATRIX: W2: {self.W2.shape}, o1: {self.o1.shape}, B2: {self.b2.shape}", flush = True)

      self.a2 = np.matmul(self.W2,  self.o1) + self.b2[:, np.newaxis]
      self.o2 = self.h2(self.a2) #apply the ReLu function

      #print(f"MATRIX: W3: {self.W3.shape}, o2: {self.o2.shape}, B1: {self.b3.shape}", flush = True)

      self.a3 = np.matmul(self.W3,  self.o2) + self.b3[:, np.newaxis]
      self.o3 = self.h3(self.a3) #apply the sigmoid function

      return np.copy(self.o3)

    def backward(self, Ybatch):
      self.Ybatch = np.copy(Ybatch)

      #store the gradient of the weights as an instance variables
      J3 = self.dloss(self.o3) * self.dh3(self.a3)

      self.dW3 = -np.dot(J3, self.o2.T)
      self.db3 = -np.sum(J3, axis=1)

      J2 = np.dot(self.W3.T, J3) * self.dh2(self.a2)

      self.dW2 = -np.dot(J2, self.o1.T)
      self.db2 = -np.sum(J2, axis=1)

      J1 = np.dot(self.W2.T, J2) * self.dh1(self.a1)

      self.dW1 = -np.dot(J1, self.Xbatch.T)
      self.db1 = -np.sum(J1, axis=1)

    def adam_step(self, alpha=0.0001, rho1=0.0, rho2=0.999, delta=1e-8):
      t = self.timeStep

      #compute for W3
      self.nuP_W3 = rho1 * self.nuP_W3 + (1 - rho1) * self.dW3
      self.hP_W3 = rho2 * self.hP_W3 + ((1 - rho2) * self.dW3) * self.dW3

      nuPhat_W3 = self.nuP_W3/(1 - rho1**t)
      hPhat_W3 = self.hP_W3/(1-rho2**t)

      self.W3 = self.W3 - (alpha * nuPhat_W3)/(np.sqrt(hPhat_W3) + delta)

      #compute for b3
      self.nuP_b3 = rho1 * self.nuP_b3 + (1 - rho1) * self.db3
      self.hP_b3 = rho2 * self.hP_b3 + ((1 - rho2) * self.db3) * self.db3

      nuPhat_b3 = self.nuP_b3/(1 - rho1**t)
      hPhat_b3 = self.hP_b3/(1-rho2**t)

      self.b3 = self.b3 - (alpha * nuPhat_b3)/(np.sqrt(hPhat_b3) + delta)

      #compute for W2
      self.nuP_W2 = rho1 * self.nuP_W2 + (1 - rho1) * self.dW2
      self.hP_W2 = rho2 * self.hP_W2 + ((1 - rho2) * self.dW2) * self.dW2

      nuPhat_W2 = self.nuP_W2/(1 - rho1**t)
      hPhat_W2 = self.hP_W2/(1-rho2**t)

      self.W2 = self.W2 - (alpha * nuPhat_W2)/(np.sqrt(hPhat_W2) + delta)

      #print("nuP_W2: {}\nhP_W2: {}\nnuPhat_W2: {}\nhPhat_W2: {}\n\n".format(self.nuP_W2.shape, self.nhP_W2.shape, ))

      #compute for b2
      self.nuP_b2 = rho1 * self.nuP_b2 + (1 - rho1) * self.db2
      self.hP_b2 = rho2 * self.hP_b2 + ((1 - rho2) * self.db2) * self.db2

      nuPhat_b2 = self.nuP_b2/(1 - rho1**t)
      hPhat_b2 = self.hP_b2/(1-rho2**t)

      self.b2 = self.b2 - (alpha * nuPhat_b2)/(np.sqrt(hPhat_b2) + delta)

      #compute for W1
      self.nuP_W1 = rho1 * self.nuP_W1 + (1 - rho1) * self.dW1
      self.hP_W1 = rho2 * self.hP_W1 + ((1 - rho2) * self.dW1) * self.dW1

      nuPhat_W1 = self.nuP_W1/(1 - rho1**t)
      hPhat_W1 = self.hP_W1/(1-rho2**t)

      self.W1 = self.W1 - (alpha * nuPhat_W1)/(np.sqrt(hPhat_W1) + delta)

      #compute for b1
      self.nuP_b1 = rho1 * self.nuP_b1 + (1 - rho1) * self.db1
      self.hP_b1 = rho2 * self.hP_b1 + ((1 - rho2) * self.db1) * self.db1

      nuPhat_b1 = self.nuP_b1/(1 - rho1**t)
      hPhat_b1 = self.hP_b1/(1-rho2**t)

      self.b1 = self.b1 - (alpha * nuPhat_b1)/(np.sqrt(hPhat_b1) + delta)

      #iterate the time step
      self.timeStep += 1

    def train(self, training_data, num_epochs, batch_size, learning_rate):
      start_time = time.time()

      #transform the training data into a numpy
      training_tensor = torch.tensor(training_data.values)

      for epoch_idx in range(num_epochs):
        print(f"EPOCH: {epoch_idx + 1}")

        epoch = training_tensor[torch.randperm(training_tensor.size(0))] #generate a random permutation of X rows
        total_loss = 0

        for batch_idx in range(batch_size, epoch.size(0), batch_size):
          #generate the minibatch
          start, end = batch_idx - batch_size, batch_idx
          X_minibatch, y_minibatch = epoch[start:end, 1:].float().numpy().T, epoch[start:end, 0]

          #y_minibatch is just a tensor of labels, we need to one hot encode them as tensors
          y_encoded = torch.nn.functional.one_hot(y_minibatch, 10).float().numpy().T

          # Compute prediction and loss on the minibatch
          self.forward(X_minibatch)

          loss = self.loss(X_minibatch, y_encoded)
          total_loss += loss

          # Backpropagation
          self.backward(y_encoded)
          self.adam_step()

          #print the batch loss
          print("\tminibatch {:<3d} LOSS: {:.2f}   [{:5d}/{:5d}]".format(int (batch_idx/batch_size), loss, end, epoch.size(0)))

        #print the average loss for the epoch
        print("\n\tAVG. LOSS: {:.2f}".format(total_loss/(epoch.size(0) // batch_size)))


      end_time = time.time()
      print("\nTIME: {:.2f} sec".format(end_time-start_time))

    def test(self, X_test, y_test):

      #generate predictions and convert them to labels
      y_preds_probs = model1.forward(X_test.numpy().T)
      y_preds = np.argmax(y_preds_probs.T, axis=1)

      #compute the accracy
      test_accuracy = 100 * np.equal(y_test.numpy(), y_preds).sum()/y_test.size(0)
      print("MODEL 1 ACCURACY: {:.2f}%".format(test_accuracy))


I decided to use minibatch for training in this project, because thhe size of my dataset included 42000 rows, I felt that doing stochastic and batch would be too slow, and minibatch would provide a good balance of accuracy and speed.

In [156]:
model1 = NeurelNetwork(784, 1000, 1000, 10)
model1.train(df_train, 6, 2000, 0.01)

EPOCH: 1


<ipython-input-153-6163baad2e64>:113: RuntimeWarning: divide by zero encountered in log
  f = lambda x, y : -(x * np.log(y) + (1 - x) * np.log(1 - y)) #negative log likely-hood
<ipython-input-153-6163baad2e64>:113: RuntimeWarning: invalid value encountered in double_scalars
  f = lambda x, y : -(x * np.log(y) + (1 - x) * np.log(1 - y)) #negative log likely-hood
<ipython-input-153-6163baad2e64>:113: RuntimeWarning: invalid value encountered in log
  f = lambda x, y : -(x * np.log(y) + (1 - x) * np.log(1 - y)) #negative log likely-hood


	minibatch 1   LOSS: nan   [ 2000/33600]
	minibatch 2   LOSS: nan   [ 4000/33600]
	minibatch 3   LOSS: nan   [ 6000/33600]
	minibatch 4   LOSS: nan   [ 8000/33600]
	minibatch 5   LOSS: nan   [10000/33600]
	minibatch 6   LOSS: nan   [12000/33600]
	minibatch 7   LOSS: nan   [14000/33600]
	minibatch 8   LOSS: nan   [16000/33600]
	minibatch 9   LOSS: nan   [18000/33600]
	minibatch 10  LOSS: nan   [20000/33600]
	minibatch 11  LOSS: nan   [22000/33600]
	minibatch 12  LOSS: nan   [24000/33600]
	minibatch 13  LOSS: nan   [26000/33600]
	minibatch 14  LOSS: nan   [28000/33600]
	minibatch 15  LOSS: nan   [30000/33600]
	minibatch 16  LOSS: nan   [32000/33600]

	AVG. LOSS: nan
EPOCH: 2
	minibatch 1   LOSS: nan   [ 2000/33600]
	minibatch 2   LOSS: nan   [ 4000/33600]
	minibatch 3   LOSS: nan   [ 6000/33600]
	minibatch 4   LOSS: nan   [ 8000/33600]
	minibatch 5   LOSS: nan   [10000/33600]
	minibatch 6   LOSS: nan   [12000/33600]
	minibatch 7   LOSS: nan   [14000/33600]
	minibatch 8   LOSS: nan   [160

In [158]:
#seperate the testing data into features and target
X_test, y_test = torch.tensor(df_test.values)[:, 1:].float(), torch.tensor(df_test.values)[:, 0]

model1.test(X_test, y_test)

MODEL 1 ACCURACY: 82.96%


#Part 2

##Task 1

https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html
https://www.datacamp.com/tutorial/pytorch-tutorial-building-a-simple-neural-network-from-scratch

These are two links that I used when I was constructing my neural network in pytorch. They First one
pretty much covered how to create a neurel network as an object, and the second link provided the information
I needed when it came to figuring out how to generate predicitions using that dataset.
Any remaining research beyond these two linked pretty much came from browsing the pytorch
library for various functions to make programming, especially the testing portion, easier.
The link to the pytorch library is as follows...

https://pytorch.org/docs/stable/index.html

##Task 2

In [ ]:
import torch
from torch import nn
from torch import optim

import time

class NeuralNetwork2(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(784, 1000),
            nn.ReLU(),
            nn.Linear(1000, 1000),
            nn.ReLU(),
            nn.Linear(1000, 10),
            nn.Sigmoid()
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

def train(model, training_data, num_epochs, batch_size, learning_rate):
  start_time = time.time()

  #initalize the loss function and the optimizer
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

  #transform the training data into a tensor
  training_tensor = torch.tensor(training_data.values)

  for epoch_idx in range(num_epochs):
    print(f"EPOCH: {epoch_idx + 1}")

    epoch = training_tensor[torch.randperm(training_tensor.size(0))] #generate a random permutation of X rows
    total_loss = 0

    for batch_idx in range(batch_size, epoch.size(0), batch_size):
      #generate the minibatch
      start, end = batch_idx - batch_size, batch_idx
      X_minibatch, y_minibatch = epoch[start:end, 1:].float(), epoch[start:end, 0]

      #y_minibatch is just a tensor of labels, we need to one hot encode them as tensors
      y_encoded = torch.nn.functional.one_hot(y_minibatch, 10).float()

      #zero the gradient and
      optimizer.zero_grad()  # Clear gradients from the previous iteration
      y_preds = model(X_minibatch)

      #compute the loss
      loss = criterion(y_preds, y_encoded)
      total_loss += loss

      #preform backwards propogation and update the weights
      loss.backward()
      optimizer.step()

      #print the batch loss
      print("\tminibatch {:<3d} LOSS: {:.2f}   [{:5d}/{:5d}]".format(int (batch_idx/batch_size), loss, end, epoch.size(0)))

    #print the average loss for the epoch
    print("\n\tAVG. LOSS: {:.2f}".format(total_loss/(epoch.size(0) // batch_size)))


  end_time = time.time()
  print("\nTIME: {:.2f} sec".format(end_time-start_time))

In [ ]:
model2 = NeuralNetwork2()
train(model2, df_train, 10, 2000, 0.01)

EPOCH: 1
	minibatch 1   LOSS: 2.36   [ 2000/33600]
	minibatch 2   LOSS: 2.17   [ 4000/33600]
	minibatch 3   LOSS: 2.08   [ 6000/33600]
	minibatch 4   LOSS: 2.09   [ 8000/33600]
	minibatch 5   LOSS: 2.07   [10000/33600]
	minibatch 6   LOSS: 1.98   [12000/33600]
	minibatch 7   LOSS: 1.97   [14000/33600]
	minibatch 8   LOSS: 1.96   [16000/33600]
	minibatch 9   LOSS: 1.84   [18000/33600]
	minibatch 10  LOSS: 1.82   [20000/33600]
	minibatch 11  LOSS: 1.79   [22000/33600]
	minibatch 12  LOSS: 1.81   [24000/33600]
	minibatch 13  LOSS: 1.77   [26000/33600]
	minibatch 14  LOSS: 1.77   [28000/33600]
	minibatch 15  LOSS: 1.75   [30000/33600]
	minibatch 16  LOSS: 1.76   [32000/33600]

	AVG. LOSS: 1.94
EPOCH: 2
	minibatch 1   LOSS: 1.74   [ 2000/33600]
	minibatch 2   LOSS: 1.76   [ 4000/33600]
	minibatch 3   LOSS: 1.74   [ 6000/33600]
	minibatch 4   LOSS: 1.74   [ 8000/33600]
	minibatch 5   LOSS: 1.71   [10000/33600]
	minibatch 6   LOSS: 1.72   [12000/33600]
	minibatch 7   LOSS: 1.72   [14000/33600

In [159]:
#seperate the testing data into features and target
X_test, y_test = torch.tensor(df_test.values)[:, 1:].float(), torch.tensor(df_test.values)[:, 0]

#generate predictions and convert them to labels
y_preds_probs = model2(X_test)
y_preds = torch.argmax(y_preds_probs, dim=1)

#compute the accracy
test_accuracy = 100 * torch.eq(y_test, y_preds).sum()/y_test.size(0)
print("MODEL 2 ACCURACY: {:.2f}%".format(test_accuracy))

MODEL 2 ACCURACY: 91.67%


## Task 3

When it came to selecting the hyperparameters I went ahead and used standard stochastic gradient descent
mainly because I wanted to see how it contrasted to my implementation of the adam optimized. For the learning rate I
just chose 0.01 as it is pretty much the standard learning rate, at least according to most sources online and my
previous experience implementing neural networks in other classes. For the dimension of the two hidden layers, I used
1000 twice because I thought that that the hidden layers should have a dimension higher than the number of pixels to
try and capture more complexity than would be contained in just the pixels. As for activation functions, I used ReLu and
Sigmoid. My hope with ReLu was that it would alleviate any problem with vanishing gradient. As for sigmoid, because I was
running a classication problem, I needed the sigmoid function in order to return probabilities. For this project I did not
use regularization because my initial implementation did not incorporate it and ended up preforming fine. I also did not want
to risk the added time of preforming regularization. Finally, while it is not a hyperperameter, I used minibatch on the training
because of large size of the dataset, my hope was that it would minimize the time it took to train the model, while balancing
accuracy.